##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Classificazione di testo con TensorFlow Hub: Recensioni di film

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/text_classification_with_hub"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Visualizza su TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/it/tutorials/keras/text_classification_with_hub.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Esegui in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/it/tutorials/keras/text_classification_with_hub.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Visualizza il sorgente su GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/it/tutorials/keras/text_classification_with_hub.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Scarica il notebook</a>
  </td>
</table>

Note: La nostra comunità di Tensorflow ha tradotto questi documenti. Poichè queste traduzioni sono *best-effort*, non è garantito che rispecchino in maniera precisa e aggiornata la [documentazione ufficiale in inglese](https://www.tensorflow.org/?hl=en). 
Se avete suggerimenti per migliorare questa traduzione, mandate per favore una pull request al repository Github [tensorflow/docs](https://github.com/tensorflow/docs). 
Per proporsi come volontari alla scrittura o alla review delle traduzioni della comunità contattate la 
[mailing list docs@tensorflow.org](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs).

Questo notebook classifica recensioni di film come *positive* o *negative* usando il testo delle revisioni. Questo è un esempio di classificazione *binaria*—o a due classi, un importante tipo di problema di machine learning largamente applicabile.

Il tutorial dimostra l'applicazione di base del trasferimento di conoscenza (transfer learning n.d.t) con TensorFlow Hub e Keras.

Useremo il [dataset IMDB](https://www.tensorflow.org/datasets/catalog/imdb_reviews) che contiene il testo di 50.000 recensioni di film dall'[Internet Movie Database](https://www.imdb.com/). Esse sono divise in 25,000 recensioni per l'addestramento e 25,000 revisioni per la verifica. Gli insiemi di addestramento e verifica sono *bilanciati*, nel senso che essi contengono un eguale numero di recensioni positive e negative.

Questo notebook usa [tf.keras](https://www.tensorflow.org/guide/keras), un'API di alto livello per costruire ed addestrare modelli in TensorFlow, e [TensorFlow Hub](https://www.tensorflow.org/hub), una libreria ed una piattaforma per il trasferimento di conoscenza. Per un tutorial più avanzato di classificazione del testo che usa `tf.keras`, vedere la [MLCC Text Classification Guide](https://developers.google.com/machine-learning/guides/text-classification/).

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

!pip install tensorflow-hub
!pip install tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

## Scarichiamo il dataset IMDB

Il dataset IMDB è disponibile su [imdb reviews](https://www.tensorflow.org/datasets/catalog/imdb_reviews) o su [TensorFlow datasets](https://www.tensorflow.org/datasets). Il codice seguente scarica il dataset IMDB sulla vostra macchina (o eseguibile colab):

In [ ]:
# Split the training set into 60% and 40%, so we'll end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

## Esploriamo i dati

Prendiamoci un momento per capire la struttura dei dati. Ogni esempio è una frase che rappresenta la recensione di un film e l'etichetta corrispondente. La frase non è pre-elaborata in alcun modo. L'etichetta è un valore intero tra 0 e 1, ove 0 è una recensione negativa, ed 1 è una recensione positiva.

Stampiamo i primi 10 esempi.

In [ ]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

E stampiamo anche le prime 10 etichette.

In [ ]:
train_labels_batch

## Costruiamo il modello

La rete neurale viene creata impilando livelli—ciò richiede tre decisioni architetturali principali:

* Come rappresentare il testo?
* Quanti livelli usare nel modello?
* Quante *unità nascoste* usare per ogni livello?

In questo esempio, i dati di input sono frasi. Le etichette da predire sono 0 o 1.

Un modo di rappresentare il testo è convertire le frasi in vettori di assorbimento (embedding n.d.t.). Possiamo usare un (livello n.d.t) di assorbimento del testo pre-addestrato come primo livello, ed avremo tre vantaggi:

*   non dovremo preoccuparci della pre-elaborazione del testo,
*   potremo beneficiare del trasferimento di conoscenza,
*   il risultato dell'assorbimento ha una lunghezza fissa, e quindi sarà più semplice da elaborare.

Per questo esempio useremo un **modello di assorbimento pre-addestrato** preso da [TensorFlow Hub](https://www.tensorflow.org/hub) e chiamato [google/tf2-preview/gnews-swivel-20dim/1](https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1).

Qui ci sono altri tre modelli pre-addestrati, da provare, che rendono ancora più interessante questo tutorial:

* [google/tf2-preview/gnews-swivel-20dim-with-oov/1](https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim-with-oov/1) - lo stesso di [google/tf2-preview/gnews-swivel-20dim/1](https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1), ma con il 2.5% del vocabolario convertito in OOV buckets. Ciò può essere d'aiuto se il vocabolario del nostro caso e il vocabolario del modello non coincidono completamente.
* [google/tf2-preview/nnlm-en-dim50/1](https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1) - Un modello più ampio con un vocabolario da ~1M di vocaboli e 50 dimensioni.
* [google/tf2-preview/nnlm-en-dim128/1](https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1) - Un modello ancora più ampio con un vocabolario da ~1M di vocaboli e 128 dimensioni.

Per prima cosa creiamo un livello Keras che usa un modello TensorFlow Hub per eseguire l'assorbimento (embedding n.d.t) delle frasi, e proviamolo su una coppia di esempi di input. Notate che, indipendentemente dalla lunghezza del testo di input, il formato dell'output dell'assorbimento è: `(num_examples, embedding_dimension)`.

In [ ]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

Ora costruiamo il modello completo:

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

I livelli sono impilati sequenzialmente per realizzare il classificatore:

1. Il primo livello è un livello TensorFlow Hub. Questo livello usa un modello Saved Model pre-addestrato per mappare una frase nel suo vettore di assorbimento. Il modello di assorbimento del testo pre-addestrato che useremo ([google/tf2-preview/gnews-swivel-20dim/1](https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1)) divide le frasi in token, assorbe ciascun token e poi combina gli assorbimenti. Le dimensioni risultanti sono: `(num_examples, embedding_dimension)`.
2. Questo vettore di output di lunghezza fissa è passato attraverso un livello completamente connesso (`Denso`) con 16 unità nascoste.
3. L'ultimo livello è connesso densamente con un singolo nodo di output. Usando la funzione di attivazione `sigmoid`, questo valore è un float tra 0 e 1, che rappresenta una probabilità, o un livello di confidenza.

Compiliamo il modello.

### Funzione obiettivo e ottimizzatore

Un modello, per l'addestramento, ha bisogno di una funzione obiettivo e di un ottimizzatore. Essendo questo un problema di classificazione binaria e l'output del modello una probabilità (un livello a unità singola con un'attivazione sigmoid), useremo la funzione obiettivo `binary_crossentropy`.

Questa non è l'unica scelta possibile per una funzione obiettivo, potreste, per esempio, scegliere la `mean_squared_error`. In generale, però, `binary_crossentropy` è migliore per gestire probabilità—essa misura la "distanza" tra distribuzioni di probabilità o, nel nostro caso, tra la distribuzione dei dati reali e le previsioni.

Nel seguito, quando esploreremo i problemi di regressione (diciamo, per prevedere il prezzo di una casa), vedremo come usare un'altra funzione obiettivo chiamata scarto quadratico medio.

Adesso, configuriamo il modello per usare un ottimizzatore ed una funzione obiettivo:

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Addestriamo il modello

Addestriamo il modello per 20 epoche in mini-lotti di 512 campioni l'uno. Cioè 20 iterazioni su tutti i campioni nei tensori `x_train` e `y_train`. Durante l'addestramento, controlliamo l'obiettivo e l'accuratezza del modello sui 10.000 campioni dell'insieme di validazione:

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

## Valutiamo il modello

E andiamo a vedere come si comporta il modello. Saranno restituiti due valori. Obietttivo (un numero che rappresenta il nostro errore, per cui valori piccoli sono migliori), e accuratezza.

In [ ]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

Questo approccio abbastanza ingenuo raggiunge un'accuratezza di circa l'87%. Con approcci più avanzati, il modello potrebbe avvicinarsi al 95%.

## Ulteriori letture

Per un metodo più generale di trattamento degli input stringa e per un'analisi più dettagliata del progresso di accuratezza e obiettivo durante l'addestramento, date un'occhiata [qui](https://www.tensorflow.org/tutorials/keras/basic_text_classification).